In [1]:
% cd /home/mayu-ot/durga/Experiments/loc_iparaphrasing/

/mnt/fs1/mayu-ot/Experiments/loc_iparaphrasing


In [2]:
import pandas as pd
import numpy as np
import tables

In [3]:
def save_plclc_roi_feat(split):
    h5file = tables.open_file('data/region_feat/frcnn_full/%s.h5' % split)
    df = pd.read_csv('data/pl-clc/phrase_pair_wt_plclcbbox_%s.csv' % split)

    roi_meta1 = pd.DataFrame({'image': df.image, 'roi': df.roi1})
    roi_meta2 = pd.DataFrame({'image': df.image, 'roi': df.roi2})
    roi_meta = pd.concat([roi_meta1, roi_meta2])

    roi_meta_unique = roi_meta.drop_duplicates()
    roi_meta_unique.index = np.arange(len(roi_meta_unique))

    feat_arr = np.zeros((len(roi_meta_unique), 4096), dtype=np.float32)

    for i, row in roi_meta_unique.iterrows():
        image = row.image
        roi = row.roi
        arr = h5file.get_node('/', str(image))
        feat_arr[i] = arr[roi]

    np.save('data/region_feat/pl-clc-frcnn/%s' % split, feat_arr)
    roi_meta_unique.to_csv('data/region_feat/pl-clc-frcnn/%s.csv' % split)

In [4]:
save_plclc_roi_feat('val')
save_plclc_roi_feat('test')
save_plclc_roi_feat('train')